<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Images" data-toc-modified-id="Images-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Images</a></span></li><li><span><a href="#Texts" data-toc-modified-id="Texts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Texts</a></span></li></ul></div>

In [1]:
import os
import random
import functools
from functools import partial
import PIL

import numpy as np 
import pandas as pd

from tqdm.notebook import tqdm
import math

import torch
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertModel

import timm
np.random.seed(1337)


## Images

In [2]:
class SoftMaxDS(Dataset):
    def __init__(self, data, images_path, return_triplet = True):
        super().__init__()
        self.imgs = data['image'].tolist()
        self.unique_labels = data['label_group'].unique().tolist()
        self.labels = data['label_group'].astype('category')
        self.label_codes = self.labels.cat.codes
        
        self.images_path = images_path
        
    def __getitem__(self, idx):
        
        img = self._get_item(idx)
        label = self.label_codes.iloc[idx]
        return img, label
    def __len__(self):
        return len(self.imgs)
    
    def _get_item(self, idx):
        im = PIL.Image.open(os.path.join(self.images_path, self.imgs[idx]))
        im = torch.tensor(np.array(im) / 255.0, dtype = torch.float).permute(2,0,1)
        return im

In [3]:
# load in data

df = pd.read_csv('data/train.csv')
small_images_dir = 'data/small_train_images/'
n_classes = df['label_group'].nunique()

labels = np.random.permutation(df['label_group'].unique())

train_perc = 0.7
train_idx = int(train_perc * len(labels))

train_labels = labels[:train_idx]
val_labels = labels[train_idx:]

In [4]:
train_df = df[df['label_group'].isin(train_labels)]
val_df = df[df['label_group'].isin(val_labels)]

In [5]:
# creating dataloaders

vision_model = 'resnet50'

bs = 64
tr_ds = SoftMaxDS(train_df, small_images_dir)
tr_dl = DataLoader(tr_ds, batch_size = bs, shuffle = True, pin_memory = True)

val_ds = SoftMaxDS(val_df, small_images_dir)
val_dl = DataLoader(val_ds, batch_size = bs, shuffle = False, pin_memory = True)
device = torch.device('cuda')

val_label_count = val_df['label_group'].value_counts()

In [6]:
class EMBRes(nn.Module) :
    def __init__(self, pretrained_image_embedor='resnet50',
                output_dim=512) :
        super(EMBRes, self).__init__()
        self.image_embedor = timm.create_model(pretrained_image_embedor, pretrained=True)
        self.image_pool = nn.AdaptiveAvgPool2d((1,1))
        self.head = nn.Sequential(nn.Linear(2048, output_dim), 
                                  #nn.ReLU(),
                                  )
        
        for m in self.head.modules():
            if isinstance(m, nn.Linear):
                sz = m.weight.data.size(-1)
                m.weight.data.normal_(mean=0.0, std=1/np.sqrt(sz))
            elif isinstance(m, (nn.LayerNorm, nn.BatchNorm1d)):
                m.bias.data.zero_()
                m.weight.data.fill_(1.0)
                m.bias.data.zero_()
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()
    
    def _get_embs(self, x) :
        images = x
        out_images = self.image_embedor.forward_features(images)
        out_images = self.image_pool(out_images).squeeze()
        #return F.normalize(out_images, dim=-1)
        return out_images
    
    def forward(self, x) :
        out_images = self._get_embs(x)
        
        return self.head(out_images)

In [7]:
# https://github.com/ronghuaiyang/arcface-pytorch

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output


***Embeddings normalization is not done in the model but in the arcface metric***

In [8]:
model = EMBRes(vision_model).to(device)
metric_fc = ArcMarginProduct(512, df['label_group'].nunique(), s=30, m=0.5, easy_margin=False).to(device)

In [9]:

normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))

train_transforms = transforms.Compose([transforms.ColorJitter(.3,.3,.3),
                                       transforms.RandomRotation(5),
                                       transforms.RandomCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       normalize
                                       ])

val_transforms = transforms.Compose([normalize
                                     ])

n_epochs = 30

lf = nn.CrossEntropyLoss()

lr = 1e-2
wd = 0
no_decay = ["bias", "BatchNorm2d.weight", "BatchNorm2d.bias", "LayerNorm.weight", 'LayerNorm.bias',
            "BatchNorm1d.weight", "BatchNorm1d.bias"]

params = list(model.named_parameters()) + list(metric_fc.named_parameters())
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in params if not any(nd in n for nd in no_decay)],
        "weight_decay": wd,
    },
    {
        "params": [p for n, p in  model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr)

# learning rate scheduler
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr =lr, pct_start = 0.3, #anneal_strategy = 'linear',
                                            total_steps = int(n_epochs * len(tr_dl)))

In [10]:
def compute_f1(embeddings, ls, threshold) :
    dists = torch.cdist(embeddings, embeddings)
    
    print('Computing distances...')
    distances, indices = torch.topk(dists, 50, dim=1, largest=False)
    
    print('Making predictions...')
    THRESHOLD = threshold
    preds = [[] for _ in range(embeddings.shape[0])]
    for i in tqdm(range(distances.shape[0])) :
        IDX = torch.where(distances[i,]<THRESHOLD)[0]
        IDS = indices[i,IDX]
        preds[i] = IDS.cpu().numpy()
            
    print('Computing f1 score...')
    label_counts = ls.value_counts()
    f_score = 0 
    for i in tqdm(range(embeddings.shape[0])) :
        tp = 0
        fp = 0
        true_label = ls.iloc[i]
        for index in preds[i] :
            if ls.iloc[index] == true_label :
                tp += 1
            else :
                fp += 1
        fn = label_counts[true_label] - tp
        #print(label_counts[true_label]-1, tp)
        f_score += 2*tp / (label_counts[true_label] + len(preds[i]))
    f_score = f_score/embeddings.shape[0]
    print('f1 score : {}'.format(f_score))
    return f_score, preds, distances, indices

In [ ]:
tr_losses = []
val_scores = []
thr = 0.6
prev_best_f_score = -100000
for ep in tqdm(range(n_epochs)):
    
    model.train()
    tr_loss = []
    pbar = tqdm(tr_dl)
    for imgs, labels in pbar:
        
        imgs = train_transforms(imgs.to(device))
        
        optimizer.zero_grad()
        feature = model(imgs)
        labels = labels.long().to(device)
        out = metric_fc(feature, labels)
        loss = lf(out, labels)
            
        loss.backward()
        optimizer.step()
        sched.step()
        
        tr_loss.append(loss.item())
        pbar.set_description(f"Train loss: {round(np.mean(tr_loss),3)}")
    
    if ep%2==0 :
        torch.save(model.state_dict(), 'data/tests_model_image/model_arcf_split_res50_ep_{}.pth'.format(ep))
    
    model.eval()
    with torch.no_grad():
        pbar = tqdm(val_dl)
        embs = []
        for imgs, _ in pbar:
            imgs = val_transforms(imgs.to(device))
            feature = model(imgs)
            embs.append(feature)
        embs = F.normalize(torch.cat(embs, 0))
    
    f_score = compute_f1(embs, val_df['label_group'], thr)[0]
    if f_score > prev_best_f_score :
        prev_best_f_score = f_score
        torch.save(model.state_dict(), 'data/tests_model_image/model_arcf_split_res50_best.pth'.format(ep))
        print('Saved best model ep {} with f score : {} for thresh : {}'.format(ep, vl, thr))
    
    
    tr_losses.append(tr_loss)
    val_scores.append(f_score)
    summary = f"Ep {ep}: Train loss {np.asarray(tr_loss).mean()} | Val f score {f_score}"
    print(summary) 
    

In [15]:
#torch.save(model.state_dict(), 'data/tests_model_image/model_class_ep_30.pth')

In [9]:
model.load_state_dict(torch.load('data/tests_model_image/model_class_ep_30.pth'))

<All keys matched successfully>

In [10]:
testing_ds = SoftMaxDS(df, small_images_dir)
testing_dl = DataLoader(testing_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [11]:
normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))

train_transforms = transforms.Compose([transforms.ColorJitter(.3,.3,.3),
                                       transforms.RandomRotation(5),
                                       transforms.RandomCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       normalize
                                       ])

val_transforms = transforms.Compose([normalize
                                     ])

In [12]:
embs = []
model.eval()
with torch.no_grad():
    pbar = tqdm(testing_dl)
    for image, labels in pbar:
        x = val_transforms(image.to(device))
        y = model(x)
        embs.append(y.cpu())

In [13]:
embs = F.normalize(torch.cat(embs,0))

In [14]:
embs_df = pd.DataFrame(embs.numpy())
emb_cols = [f'emb_{i}' for i in embs_df.columns]
embs_df.columns = emb_cols
embs_df.to_csv('data/tests_model_image/train_embs_arcf_30ep.csv')


## Texts

In [1]:
import os
import random
import functools
from functools import partial
import PIL

import numpy as np 
import pandas as pd

from tqdm.notebook import tqdm
import math

import torch
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import BertModel

import timm

In [2]:
class SoftMaxDS(Dataset):
    def __init__(self, data, tokenizer):
        super().__init__()
        self.unique_labels = data['label_group'].unique().tolist()
        self.labels = data['label_group'].tolist()
        self.label_to_index_dict = (data.reset_index(drop = True)
                                    .groupby('label_group')
                                    .apply(lambda x: x.index.tolist())
                                    .to_dict())
        self.texts = tokenizer(data['title'].values.tolist(), return_tensors = 'pt',
                               padding=True, truncation=True, max_length = 40)
        
        
    def __getitem__(self, idx):
        
        # anchor data
        label = self.labels[idx]
        txt = self._get_item(idx)
        
        return txt, label
        
        
    def __len__(self):
        return len(self.labels)
    
    def _get_item(self, idx):
        txt = {'input_ids' : self.texts['input_ids'][idx], 
               'attention_mask' : self.texts['attention_mask'][idx]}
        return txt

In [3]:
# load in data

df = pd.read_csv('data/train.csv')
small_images_dir = 'data/small_train_images/'
n_classes = df['label_group'].nunique()
np.random.seed(1337)

# train val split

train_perc = 0.7
n_train_examples = int(train_perc * len(df))

train_df = df.iloc[:n_train_examples]
val_df = df.iloc[n_train_examples:]

In [4]:
# creating dataloaders

language_model = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(language_model)

bs = 64
tr_ds = SoftMaxDS(df, tokenizer)
tr_dl = DataLoader(tr_ds, batch_size = bs, shuffle = True, pin_memory = True)

device = torch.device('cuda')

In [5]:
class EMBBert(nn.Module) :
    def __init__(self, pretrained_text_embedor='bert-base-uncased',
                output_dim=512) :
        super(EMBBert, self).__init__()
        self.text_embedor = BertModel.from_pretrained(pretrained_text_embedor)
        self.head = nn.Sequential(nn.Linear(768, output_dim), 
                                  #nn.ReLU(), 
                                  #nn.Linear(1024, output_dim)
                                 )
        
        for m in self.head.modules():
            if isinstance(m, nn.Linear):
                sz = m.weight.data.size(-1)
                m.weight.data.normal_(mean=0.0, std=1/np.sqrt(sz))
            elif isinstance(m, (nn.LayerNorm, nn.BatchNorm1d)):
                m.bias.data.zero_()
                m.weight.data.fill_(1.0)
                m.bias.data.zero_()
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()
            
    
    def forward(self, x) :
        texts = x
        
        out_text = self.text_embedor(texts['input_ids'], 
                                     attention_mask=texts['attention_mask'])[0][:,0,:]
        return F.normalize(self.head(out_text), dim=-1)

In [6]:
# https://github.com/ronghuaiyang/arcface-pytorch

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output


***Embeddings normalization is not done in the model but in the arcface metric***

In [7]:
model = EMBBert().to(device)
metric_fc = ArcMarginProduct(512, df['label_group'].nunique(), s=30, m=0.5, easy_margin=False).to(device)

In [8]:
def text_to_device(text, device):
    return {'input_ids' : text['input_ids'].to(device),
            'attention_mask' : text['attention_mask'].to(device)}

In [9]:
n_epochs = 30

lf = nn.CrossEntropyLoss()

lr = 1e-2
wd = 0
no_decay = ["bias", "BatchNorm2d.weight", "BatchNorm2d.bias", "LayerNorm.weight", 'LayerNorm.bias',
            "BatchNorm1d.weight", "BatchNorm1d.bias"]

params = list(model.named_parameters()) + list(metric_fc.named_parameters())
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in params if not any(nd in n for nd in no_decay)],
        "weight_decay": wd,
    },
    {
        "params": [p for n, p in  model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr)

# learning rate scheduler
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr =lr, pct_start = 0.3, #anneal_strategy = 'linear',
                                            total_steps = int(n_epochs * len(tr_dl)))

In [10]:
txt = next(iter(tr_dl))

In [ ]:
txt

In [20]:
tr_losses = []
val_losses = []
for ep in tqdm(range(n_epochs)):
    model.train()
    tr_loss = []
    pbar = tqdm(tr_dl)
    for txts in pbar:
        
        txts = text_to_device(txts, device)
        
        optimizer.zero_grad()
        feature = model(txts)
        labels = labels.long().to(device)
        out = metric_fc(feature, labels)
        loss = lf(out, labels)
            
        loss.backward()
        optimizer.step()
        sched.step()
        
        tr_loss.append(loss.item())
        pbar.set_description(f"Train loss: {round(np.mean(tr_loss),3)}")
    
    if ep%2==0 :
        torch.save(model.state_dict(), 'data/tests_model_image/model_arcfsplit_ep_{}.pth'.format(ep))
    model.eval()
    val_loss = []
    with torch.no_grad():
        pbar = tqdm(val_dl)
        for anchor_image, anchor_text, pos_image, pos_text, neg_image, neg_text in pbar:

            anchor = val_transforms(anchor_image.to(device)), text_to_device(anchor_text, device)
            pos = val_transforms(pos_image.to(device)), text_to_device(pos_text, device)
            neg = val_transforms(neg_image.to(device)), text_to_device(neg_text, device)

            with torch.cuda.amp.autocast():
                
                anchor_emb = model(anchor)
                pos_emb = model(pos)
                neg_emb = model(neg)
                loss = lf(anchor_emb, pos_emb, neg_emb)

            val_loss.append(loss.item())
            pbar.set_description(f"Val loss: {round(np.mean(val_loss),3)}")
    tr_losses.append(tr_loss)
    summary = f"Ep {ep}: Train loss {np.asarray(tr_loss).mean()} | Val loss {np.asarray(val_loss).mean()}"
    print(summary) 
    

TypeError: list indices must be integers or slices, not str

In [21]:
%debug

> <ipython-input-17-f4f7064a1f00>(2)text_to_device()
      1 def text_to_device(text, device):
----> 2     return {'input_ids' : text['input_ids'].to(device),
      3             'attention_mask' : text['attention_mask'].to(device)}

ipdb> text
[{'input_ids': tensor([[  101, 14163,  7520,  ...,     0,     0,     0],
        [  101,  9152,  3726,  ...,     0,     0,     0],
        [  101, 18906,  5575,  ...,     0,     0,     0],
        ...,
        [  101,  6090, 15444,  ...,     0,     0,     0],
        [  101,  3681, 26348,  ...,     0,     0,     0],
        [  101,  1031, 19337,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}, tensor([ 420878019, 3639207904,  302466871, 4245771399, 3157557152, 1322597928,
          57219072,  159351600, 3047779462,  574325131, 122

In [15]:
#torch.save(model.state_dict(), 'data/tests_model_image/model_class_ep_30.pth')

In [9]:
model.load_state_dict(torch.load('data/tests_model_image/model_class_ep_30.pth'))

<All keys matched successfully>

In [10]:
testing_ds = SoftMaxDS(df, small_images_dir)
testing_dl = DataLoader(testing_ds, batch_size = bs, shuffle = False, pin_memory = True)

In [11]:
normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))

train_transforms = transforms.Compose([transforms.ColorJitter(.3,.3,.3),
                                       transforms.RandomRotation(5),
                                       transforms.RandomCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       normalize
                                       ])

val_transforms = transforms.Compose([normalize
                                     ])

In [12]:
embs = []
model.eval()
with torch.no_grad():
    pbar = tqdm(testing_dl)
    for image, labels in pbar:
        x = val_transforms(image.to(device))
        y = model(x)
        embs.append(y.cpu())

In [13]:
embs = F.normalize(torch.cat(embs,0))

In [14]:
embs_df = pd.DataFrame(embs.numpy())
emb_cols = [f'emb_{i}' for i in embs_df.columns]
embs_df.columns = emb_cols
embs_df.to_csv('data/tests_model_image/train_embs_arcf_30ep.csv')
